In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

from scipy.stats import spearmanr

from uncertain_x.line_fit import uncertain_x

In [2]:
%matplotlib notebook

# Volume Limited Sample of Normal SNe Ia

This notebook provides summary statistics when examining a volume limited sample of "normal" SNe Ia from the ZTF 2018 early sample.

In [3]:
info_path="../../forced_lightcurves/sample_lc_v2/"
salt_df = pd.read_csv(info_path + "../../Nobs_cut_salt2_spec_subtype_pec.csv")
res = pd.read_csv('results_40percent.csv')
res_tsquared = pd.read_csv('results_40_tsquared.csv')

In [4]:
normal = np.where((salt_df['Ia subtype'] != 'Ia-CSM') & 
                  (salt_df['Ia subtype'] != 'SC') & 
                  (salt_df['Ia subtype'] != 'SC*') & 
                  (salt_df['Ia subtype'] != '86G-like') & 
                  (salt_df['Ia subtype'] != '02cx-like'))
good = np.where(res.final_selection.values == 1)
has_host_z = np.where(salt_df.z_source != 'Msl')

normal_good = np.intersect1d(normal, good)
normal_good_host = np.intersect1d(normal_good, has_host_z)

# rearange for plotting
normal_plot = np.setdiff1d(normal[0], normal_good)
normal_good_plot = np.setdiff1d(normal_good, normal_good_host)

In [5]:
# plotting dictionary
color_dict = {'orange': '#F0810F',
              'blueberry': '#063852',
              'dark navy': '#011A27'}

In [6]:
low_z = np.where(salt_df.z_adopt < 0.06)


normal_lowz = np.intersect1d(low_z, normal)
print(len(normal_lowz))
normal_good_lowz = np.intersect1d(low_z, normal_good)
print(len(normal_good_lowz))
normal_good_host_lowz = np.intersect1d(low_z, normal_good_host)
print(len(normal_good_host_lowz))

28
16
15


In [7]:
t_rise_16 = res.t_rise_16.values.copy()
t_rise_50 = res.t_rise_50.values.copy()
t_rise_84 = res.t_rise_84.values.copy()

sigma_i = (t_rise_84-t_rise_16)/2
w_i = (sigma_i)**(-2)

In [11]:
print('The following are summary statistics for rise time:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_lowz]), np.median(t_rise_50[normal_lowz]),
                                                                   np.average(t_rise_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(t_rise_50[normal_lowz] - np.median(t_rise_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

salt_uninformative = uncertain_x.UncertainAbscissa('salt')
salt_uninformative.add_data(salt_df.x1_salt2.values[normal_lowz], t_rise_50[normal_lowz], 
                            salt_df.x1_salt2_unc.values[normal_lowz], sigma_i[normal_lowz])
salt_uninformative.mle_slope_intercept(slope_guess=1.0, intercept_guess=17)
print('the slope is: {:.4f} +/- {:.4f}'.format(salt_uninformative.m, salt_uninformative.m_unc))
print('the intercept is: {:.4f} +/- {:.4f}'.format(salt_uninformative.b, salt_uninformative.b_unc))

residuals = t_rise_50[normal_lowz] - np.polyval([salt_uninformative.m, salt_uninformative.b], 
                                                salt_df.x1_salt2.values[normal_lowz])
print('the residual scatter is: {:.4f} d'.format(np.std(residuals, ddof=1)))
print(' ')
print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_lowz]), np.median(t_rise_50[normal_good_lowz]),
                                                                   np.average(t_rise_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_lowz] - np.median(t_rise_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))
salt_uninformative = uncertain_x.UncertainAbscissa('salt')
salt_uninformative.add_data(salt_df.x1_salt2.values[normal_good_lowz], t_rise_50[normal_good_lowz], 
                            salt_df.x1_salt2_unc.values[normal_good_lowz], sigma_i[normal_good_lowz])
salt_uninformative.mle_slope_intercept(slope_guess=0.85, intercept_guess=17)
print('    the slope is: {:.4f} +/- {:.4f}'.format(salt_uninformative.m, salt_uninformative.m_unc))
print('    the intercept is: {:.4f} +/- {:.4f}'.format(salt_uninformative.b, salt_uninformative.b_unc))

residuals = t_rise_50[normal_good_lowz] - np.polyval([salt_uninformative.m, salt_uninformative.b], 
                                                salt_df.x1_salt2.values[normal_good_lowz])
print('    the residual scatter is: {:.4f} d'.format(np.std(residuals, ddof=1)))
print(' ')
print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_host_lowz]), np.median(t_rise_50[normal_good_host_lowz]),
                                                                   np.average(t_rise_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_host_lowz] - np.median(t_rise_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

salt_uninformative = uncertain_x.UncertainAbscissa('salt')
salt_uninformative.add_data(salt_df.x1_salt2.values[normal_good_host_lowz], t_rise_50[normal_good_host_lowz], 
                            salt_df.x1_salt2_unc.values[normal_good_host_lowz], sigma_i[normal_good_host_lowz])
salt_uninformative.mle_slope_intercept(slope_guess=0.85, intercept_guess=17)
print('\t  the slope is: {:.4f} +/- {:.4f}'.format(salt_uninformative.m, salt_uninformative.m_unc))
print('\t  the intercept is: {:.4f} +/- {:.4f}'.format(salt_uninformative.b, salt_uninformative.b_unc))

residuals = t_rise_50[normal_good_host_lowz] - np.polyval([salt_uninformative.m, salt_uninformative.b], 
                                                salt_df.x1_salt2.values[normal_good_host_lowz])
print('\t  the residual scatter is: {:.4f} d'.format(np.std(residuals, ddof=1)))

The following are summary statistics for rise time:
mean, median, weighted mean: 16.87319, 17.03738, 18.18446
std, sMAD, sigma w_mean: 2.23480, 2.42283, 0.05104
the slope is: 1.2556 +/- 0.0446
the intercept is: 17.6953 +/- 0.0589
the residual scatter is: 1.7150 d
 
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 17.93970, 17.68516, 18.47503
    std, sMAD, sigma w_mean: 1.72830, 1.95130, 0.05318
    the slope is: 0.8556 +/- 0.0352
    the intercept is: 18.0100 +/- 0.0614
    the residual scatter is: 1.4379 d
 
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 18.09045, 18.07066, 18.51542
	  std, sMAD, sigma w_mean: 1.67654, 2.00409, 0.05356
	  the slope is: 0.8045 +/- 0.0112
	  the intercept is: 18.0564 +/- 0.1360
	  the residual scatter is: 1.4095 d


In [9]:
flat_alpha_g_16 = res.flat_alpha_g_16.values.copy()
flat_alpha_g_50 = res.flat_alpha_g_50.values.copy()
flat_alpha_g_84 = res.flat_alpha_g_84.values.copy()

sigma_i = (flat_alpha_g_84-flat_alpha_g_16)/2
w_i = (sigma_i)**(-2)

In [10]:
print('The following are summary statistics for alpha_g:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_g_50[normal_lowz]), np.median(flat_alpha_g_50[normal_lowz]),
                                                                   np.average(flat_alpha_g_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_g_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(flat_alpha_g_50[normal_lowz] - np.median(flat_alpha_g_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_g_50[normal_good_lowz]), np.median(flat_alpha_g_50[normal_good_lowz]),
                                                                   np.average(flat_alpha_g_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_g_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_g_50[normal_good_lowz] - np.median(flat_alpha_g_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))

print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_g_50[normal_good_host_lowz]), np.median(flat_alpha_g_50[normal_good_host_lowz]),
                                                                   np.average(flat_alpha_g_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_g_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_g_50[normal_good_host_lowz] - np.median(flat_alpha_g_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

The following are summary statistics for alpha_g:
mean, median, weighted mean: 2.10809, 2.21567, 2.05053
std, sMAD, sigma w_mean: 0.75758, 0.88782, 0.02759
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 2.31110, 2.33805, 2.13332
    std, sMAD, sigma w_mean: 0.53738, 0.40565, 0.02906
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 2.36384, 2.36108, 2.13994
	  std, sMAD, sigma w_mean: 0.51158, 0.42534, 0.02921


In [11]:
flat_alpha_r_16 = res.flat_alpha_r_16.values.copy()
flat_alpha_r_50 = res.flat_alpha_r_50.values.copy()
flat_alpha_r_84 = res.flat_alpha_r_84.values.copy()

sigma_i = (flat_alpha_r_84-flat_alpha_r_16)/2
w_i = (sigma_i)**(-2)

In [12]:
print('The following are summary statistics for alpha_r:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_r_50[normal_lowz]), np.median(flat_alpha_r_50[normal_lowz]),
                                                                   np.average(flat_alpha_r_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_r_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(flat_alpha_r_50[normal_lowz] - np.median(flat_alpha_r_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_r_50[normal_good_lowz]), np.median(flat_alpha_r_50[normal_good_lowz]),
                                                                   np.average(flat_alpha_r_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_r_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_r_50[normal_good_lowz] - np.median(flat_alpha_r_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))

print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_r_50[normal_good_host_lowz]), np.median(flat_alpha_r_50[normal_good_host_lowz]),
                                                                   np.average(flat_alpha_r_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_r_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_r_50[normal_good_host_lowz] - np.median(flat_alpha_r_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

The following are summary statistics for alpha_r:
mean, median, weighted mean: 1.99435, 2.01032, 1.95093
std, sMAD, sigma w_mean: 0.66800, 0.60663, 0.02783
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 2.11799, 2.13313, 2.00928
    std, sMAD, sigma w_mean: 0.51820, 0.43057, 0.02911
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 2.16755, 2.24131, 2.01793
	  std, sMAD, sigma w_mean: 0.49560, 0.40529, 0.02931


### Repeat analysis for the t^2 SNe

In [17]:
t_rise_16 = res_tsquared.t_rise_16.values.copy()
t_rise_50 = res_tsquared.t_rise_50.values.copy()
t_rise_84 = res_tsquared.t_rise_84.values.copy()

sigma_i = (t_rise_84-t_rise_16)/2
w_i = (sigma_i)**(-2)

normal = np.where((salt_df['Ia subtype'] != 'Ia-CSM') & 
                  (salt_df['Ia subtype'] != 'SC') & 
                  (salt_df['Ia subtype'] != 'SC*') & 
                  (salt_df['Ia subtype'] != '86G-like') & 
                  (salt_df['Ia subtype'] != '02cx-like'))
good = np.where(res_tsquared.final_selection.values == 1)
has_host_z = np.where(salt_df.z_source != 'Msl')

normal_good = np.intersect1d(normal, good)
normal_good_host = np.intersect1d(normal_good, has_host_z)

normal_lowz = np.intersect1d(low_z, normal)
print(len(normal_lowz))
normal_good_lowz = np.intersect1d(low_z, normal_good)
print(len(normal_good_lowz))
normal_good_host_lowz = np.intersect1d(low_z, normal_good_host)
print(len(normal_good_host_lowz))

28
27
24


In [21]:
print('The following are summary statistics for rise time:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_lowz]), np.median(t_rise_50[normal_lowz]),
                                                                   np.average(t_rise_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(t_rise_50[normal_lowz] - np.median(t_rise_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

salt_uninformative = uncertain_x.UncertainAbscissa('salt')
salt_uninformative.add_data(salt_df.x1_salt2.values[normal_lowz], t_rise_50[normal_lowz], 
                            salt_df.x1_salt2_unc.values[normal_lowz], sigma_i[normal_lowz])
salt_uninformative.mle_slope_intercept(slope_guess=1.5, intercept_guess=20)
print('the slope is: {:.4f} +/- {:.4f}'.format(salt_uninformative.m, salt_uninformative.m_unc))
print('the intercept is: {:.4f} +/- {:.4f}'.format(salt_uninformative.b, salt_uninformative.b_unc))

residuals = t_rise_50[normal_lowz] - np.polyval([salt_uninformative.m, salt_uninformative.b], 
                                                salt_df.x1_salt2.values[normal_lowz])
print('the residual scatter is: {:.4f} d'.format(np.std(residuals, ddof=1)))
print(' ')
print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_lowz]), np.median(t_rise_50[normal_good_lowz]),
                                                                   np.average(t_rise_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_lowz] - np.median(t_rise_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))
salt_uninformative = uncertain_x.UncertainAbscissa('salt')
salt_uninformative.add_data(salt_df.x1_salt2.values[normal_good_lowz], t_rise_50[normal_good_lowz], 
                            salt_df.x1_salt2_unc.values[normal_good_lowz], sigma_i[normal_good_lowz])
salt_uninformative.mle_slope_intercept(slope_guess=1.5, intercept_guess=20)
print('    the slope is: {:.4f} +/- {:.4f}'.format(salt_uninformative.m, salt_uninformative.m_unc))
print('    the intercept is: {:.4f} +/- {:.4f}'.format(salt_uninformative.b, salt_uninformative.b_unc))

residuals = t_rise_50[normal_good_lowz] - np.polyval([salt_uninformative.m, salt_uninformative.b], 
                                                salt_df.x1_salt2.values[normal_good_lowz])
print('    the residual scatter is: {:.4f} d'.format(np.std(residuals, ddof=1)))
print(' ')
print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_host_lowz]), np.median(t_rise_50[normal_good_host_lowz]),
                                                                   np.average(t_rise_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_host_lowz] - np.median(t_rise_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

salt_uninformative = uncertain_x.UncertainAbscissa('salt')
salt_uninformative.add_data(salt_df.x1_salt2.values[normal_good_host_lowz], t_rise_50[normal_good_host_lowz], 
                            salt_df.x1_salt2_unc.values[normal_good_host_lowz], sigma_i[normal_good_host_lowz])
salt_uninformative.mle_slope_intercept(slope_guess=1.5, intercept_guess=20)
print('\t  the slope is: {:.4f} +/- {:.4f}'.format(salt_uninformative.m, salt_uninformative.m_unc))
print('\t  the intercept is: {:.4f} +/- {:.4f}'.format(salt_uninformative.b, salt_uninformative.b_unc))

residuals = t_rise_50[normal_good_host_lowz] - np.polyval([salt_uninformative.m, salt_uninformative.b], 
                                                salt_df.x1_salt2.values[normal_good_host_lowz])
print('\t  the residual scatter is: {:.4f} d'.format(np.std(residuals, ddof=1)))

The following are summary statistics for rise time:
mean, median, weighted mean: 20.51776, 20.50461, 21.16798
std, sMAD, sigma w_mean: 1.45710, 2.07489, 0.02098
the slope is: 1.4714 +/- 0.0067
the intercept is: 20.9436 +/- 0.1355
the residual scatter is: 0.7141 d
 
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 20.58930, 20.52734, 21.16890
    std, sMAD, sigma w_mean: 1.43387, 2.02832, 0.02099
    the slope is: 1.4715 +/- 0.0284
    the intercept is: 20.9435 +/- 0.0300
    the residual scatter is: 0.7277 d
 
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 20.50924, 20.50461, 21.16477
	  std, sMAD, sigma w_mean: 1.47281, 2.09513, 0.02118
	  the slope is: 1.4722 +/- 0.0278
	  the intercept is: 20.9431 +/- 0.0304
	  the residual scatter is: 0.7706 d


### Repeat analysis for the mixed sample of SNe

In [30]:
dic_df = pd.read_csv('dic_results.csv')

In [31]:
no_t2 = np.where(((dic_df.dic_evidence == 'decisive') | (dic_df.dic_evidence == 'very strong')) &  
                 (res.final_selection == 1))[0]
print(len(no_t2))

13


In [32]:
t_rise_16[no_t2] = res.t_rise_16.values.copy()[no_t2]
t_rise_50[no_t2] = res.t_rise_50.values.copy()[no_t2]
t_rise_84[no_t2] = res.t_rise_84.values.copy()[no_t2]

sigma_i = (t_rise_84-t_rise_16)/2
w_i = (sigma_i)**(-2)

In [33]:
print('The following are summary statistics for rise time:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_lowz]), np.median(t_rise_50[normal_lowz]),
                                                                   np.average(t_rise_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(t_rise_50[normal_lowz] - np.median(t_rise_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_lowz]), np.median(t_rise_50[normal_good_lowz]),
                                                                   np.average(t_rise_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_lowz] - np.median(t_rise_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))

print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_host_lowz]), np.median(t_rise_50[normal_good_host_lowz]),
                                                                   np.average(t_rise_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_host_lowz] - np.median(t_rise_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

The following are summary statistics for rise time:
mean, median, weighted mean: 19.75843, 19.58847, 19.45520
std, sMAD, sigma w_mean: 1.37175, 1.35126, 0.03351
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 19.80185, 19.66088, 19.45599
    std, sMAD, sigma w_mean: 1.37814, 1.30155, 0.03352
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 19.62335, 19.51187, 19.36108
	  std, sMAD, sigma w_mean: 1.32400, 1.32875, 0.03434


### Repeat analysis for just alpha != 2 (DIC selected subsection)

In [22]:
normal_lowz = np.intersect1d(normal_lowz, no_t2)
normal_good_lowz = np.intersect1d(normal_good_lowz, no_t2)
normal_good_host_lowz = np.intersect1d(normal_good_host_lowz, no_t2)

In [23]:
print('The following are summary statistics for rise time:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_lowz]), np.median(t_rise_50[normal_lowz]),
                                                                   np.average(t_rise_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(t_rise_50[normal_lowz] - np.median(t_rise_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_lowz]), np.median(t_rise_50[normal_good_lowz]),
                                                                   np.average(t_rise_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_lowz] - np.median(t_rise_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))

print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_host_lowz]), np.median(t_rise_50[normal_good_host_lowz]),
                                                                   np.average(t_rise_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_host_lowz] - np.median(t_rise_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

The following are summary statistics for rise time:
mean, median, weighted mean: 19.09761, 19.21959, 18.83624
std, sMAD, sigma w_mean: 1.32714, 0.76041, 0.03148
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 19.09761, 19.21959, 18.83624
    std, sMAD, sigma w_mean: 1.32714, 0.76041, 0.03148
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 19.09761, 19.21959, 18.83624
	  std, sMAD, sigma w_mean: 1.32714, 0.76041, 0.03148


In [24]:
sigma_i = (flat_alpha_g_84-flat_alpha_g_16)/2
w_i = (sigma_i)**(-2)

print('The following are summary statistics for alpha_g:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_g_50[normal_lowz]), np.median(flat_alpha_g_50[normal_lowz]),
                                                                   np.average(flat_alpha_g_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_g_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(flat_alpha_g_50[normal_lowz] - np.median(flat_alpha_g_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_g_50[normal_good_lowz]), np.median(flat_alpha_g_50[normal_good_lowz]),
                                                                   np.average(flat_alpha_g_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_g_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_g_50[normal_good_lowz] - np.median(flat_alpha_g_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))

print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_g_50[normal_good_host_lowz]), np.median(flat_alpha_g_50[normal_good_host_lowz]),
                                                                   np.average(flat_alpha_g_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_g_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_g_50[normal_good_host_lowz] - np.median(flat_alpha_g_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

The following are summary statistics for alpha_g:
mean, median, weighted mean: 2.54483, 2.50204, 2.14300
std, sMAD, sigma w_mean: 0.55384, 0.36168, 0.03123
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 2.54483, 2.50204, 2.14300
    std, sMAD, sigma w_mean: 0.55384, 0.36168, 0.03123
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 2.54483, 2.50204, 2.14300
	  std, sMAD, sigma w_mean: 0.55384, 0.36168, 0.03123


In [25]:
sigma_i = (flat_alpha_r_84-flat_alpha_r_16)/2
w_i = (sigma_i)**(-2)

print('The following are summary statistics for alpha_r:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_r_50[normal_lowz]), np.median(flat_alpha_r_50[normal_lowz]),
                                                                   np.average(flat_alpha_r_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_r_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(flat_alpha_r_50[normal_lowz] - np.median(flat_alpha_r_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_r_50[normal_good_lowz]), np.median(flat_alpha_r_50[normal_good_lowz]),
                                                                   np.average(flat_alpha_r_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_r_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_r_50[normal_good_lowz] - np.median(flat_alpha_r_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))

print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_r_50[normal_good_host_lowz]), np.median(flat_alpha_r_50[normal_good_host_lowz]),
                                                                   np.average(flat_alpha_r_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_r_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_r_50[normal_good_host_lowz] - np.median(flat_alpha_r_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

The following are summary statistics for alpha_g:
mean, median, weighted mean: 2.34931, 2.42209, 2.01934
std, sMAD, sigma w_mean: 0.52445, 0.26803, 0.03148
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 2.34931, 2.42209, 2.01934
    std, sMAD, sigma w_mean: 0.52445, 0.26803, 0.03148
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 2.34931, 2.42209, 2.01934
	  std, sMAD, sigma w_mean: 0.52445, 0.26803, 0.03148
